<a href="https://colab.research.google.com/github/gloammer/Neural-networks/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn import datasets

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model

from tensorflow.keras.layers import Dense, Dropout, Flatten

from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16

In [ ]:
import cv2
import os
import glob
import gc

def lire_images(img_dir, xdim, ydim, nmax=5000) :
    label = 0
    label_names = []
    X = []
    y=[]
    for dirname in os.listdir(img_dir):
        print(dirname)
        label_names.append(dirname)
        data_path = os.path.join(img_dir + "/" + dirname,'**')
        print(data_path)
        files = [os.path.normpath(i) for i in glob.glob(data_path)]
        print(files)
        n=0
        for f1 in files:
            if n>nmax : break
            img = cv2.imread(f1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (xdim,ydim))
            X.append(np.array(img))
            y.append(label)
            n=n+1
        print(n,' images lues')
        label = label+1
    X = np.array(X)
    y = np.array(y)
    gc.collect()
    return X,y, label, label_names


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
X,y,Nombre_classes,Classes = lire_images("/content/drive/My Drive/training_set/training_set", 224, 224, 1000)

In [ ]:
Nombre_classes

In [ ]:
Classes

In [ ]:
import random
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    j = random.randint(0,len(X))
    plt.axis('off')
    plt.imshow(X[j])
    plt.title(Classes[y[j]])

In [ ]:
y = to_categorical(y)

In [ ]:
y

In [ ]:
X.shape

In [ ]:
X = X / 255

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
y_test

In [ ]:
del X,y

In [ ]:
model = Sequential()
model.add(Conv2D(128, (5, 5), input_shape=(224, 224, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(Nombre_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()


In [ ]:
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, verbose=1)

In [ ]:
# Test
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

In [ ]:

print(X_test)

In [ ]:
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Обучение')
    plt.plot(epochs, val_accuracy, 'r', label='Валидация')
    plt.title('Точность обучения')
    plt.legend()
    plt.show()

In [ ]:
plot_scores(train)

In [ ]:
predict_x=model.predict(X_test)
y_cnn=np.argmax(predict_x,axis=1)

In [ ]:
print("Верно")
plt.figure(figsize=(15,25))
n_test = X_test.shape[0]
i=1
for j in range(len(X_test)) :
    if (y_cnn[j] == y_test[j].argmax(axis=-1)) & (i<20):
        plt.subplot(5,4,i)
        plt.axis('off')
        plt.imshow(X_test[j])
        plt.title('%s / %s' % (Classes[y_cnn[j]], Classes[y_test[j].argmax(axis=-1)]))
        i+=1

In [ ]:
print("Неверно")
plt.figure(figsize=(15,25))
n_test = X_test.shape[0]
i=1
for j in range(len(X_test)) :
    if (y_cnn[j] != y_test[j].argmax(axis=-1)) & (i<20):
        plt.subplot(5,4,i)
        plt.axis('off')
        plt.imshow(X_test[j])
        plt.title('%s / %s' % (Classes[y_cnn[j]], Classes[y_test[j].argmax(axis=-1)]))
        i+=1

In [ ]:
# VGG16
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
vgg16.trainable = False

In [ ]:
vgg16.summary()


In [ ]:
model2 = Sequential()
model2.add(vgg16)
model2.add(Flatten())
model2.add(Dense(Nombre_classes, activation='softmax'))

model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model2.summary()

In [ ]:
train2 = model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, verbose=1)


In [ ]:
# Test
scores2 = model2.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores2[1]*100))

plot_scores(train2)

In [ ]:
from PIL import Image
from skimage.io import imread

def predict(file_name):
    img = Image.open(file_name)
    img = img.convert("RGB")
    img = img.resize((224, 224))
    img.save(file_name)

    plt.figure(figsize=(4, 4))
    plt.imshow(img)
    plt.axis("off")

    img = imread(file_name)
    img = tf.keras.applications.densenet.preprocess_input(img)
    img = tf.expand_dims(img, axis=0)
    probs = model2.predict(img)
    for idx in probs.argsort()[0][::-1][:5]:
        print("{:.2f}%".format(probs[0][idx]*100), "\t", Classes[idx])

def download_and_predict(url, file_name):
    os.system("curl -s {} -o {}".format(url, file_name))
    predict(file_name)

In [ ]:
download_and_predict("https://www.proplan.ru/sites/owners.proplan.ru/files/styles/article283x134/public/2020-03/alabaj-sredneaziatskaya-ovcharka_2.webp?itok=fK93lqNP",
                     "dog.jpg")

In [ ]:
download_and_predict("https://klike.net/uploads/posts/2019-06/1561011182_3.jpg",
                     "cat.jpg")

In [ ]:
download_and_predict("https://klike.net/uploads/posts/2019-06/medium/1561011184_2.jpg",
                     "cat.jpg")

In [ ]:
download_and_predict("https://www.proplan.ru/sites/owners.proplan.ru/files/styles/article283x134/public/2021-11/shutterstock_685424749_2.webp?itok=q-5TLv4u", "dog.jpg")